This notebook is used to use KIM to perform inverse modeling for ATS.

In [1]:
# Libraries
from pathlib import Path
import pandas as pd
import numpy as np

from kim.map import KIM
from kim.data import Data
from kim.mapping_model import MLP

import jax

%load_ext autoreload
%autoreload 2


In [2]:
jax.devices()

[CpuDevice(id=0)]

# Read the data

In [3]:
# File and folder paths
f_para = Path("./data/Output_para.csv")
f_state = Path("./data/Input_logq.csv")


In [4]:
df_para, df_state = pd.read_csv(f_para, index_col=0),pd.read_csv(f_state, index_col=0)

In [5]:
y_keys, x_keys = df_para.keys(), df_state.keys()
y, x = df_para.values, df_state.values

In [6]:
x.shape, y.shape

((396, 1461), (396, 8))

# Configurations

## Preliminary analysis configuration

In [7]:
seed_shuffle = 1234
f_data_save = Path("./results/data")


In [8]:
# Data configuration
data_params = {
    "xscaler_type": "minmax",
    "yscaler_type": "minmax",
}

# Sensitivity analysis configuration
sensitivity_params = {
    "method": "pc", "metric": "it-knn",
    "sst": True, "ntest": 100, "alpha": 0.05, "k": 3,
    "n_jobs": 100, "seed_shuffle": seed_shuffle,
    "verbose": 1
}


## Ensemble learning configuration

In [9]:
Ns_train = 300
Ns_val = 50
hidden_activation = 'sigmoid'
final_activation = 'leaky_relu'
seed_ens = 1024
seed_predict = 3636
seed_dl = 10
seed_model = 100
training_verbose = 1
n_models = 100
n_jobs = 20

f_kim_save1 = Path("./results/map_many2many")
f_kim_save2 = Path("./results/map_many2one")
f_kim_save3 = Path("./results/map_many2one_cond")


In [10]:
# Mapping parameters for each test below
map_configs = {
    "model_type": MLP,
    'n_model': n_models,
    'ensemble_type': 'ens_random',
    'model_hp_choices': {
        "depth": [1,3,5,6],
        "width_size": [3,6,10]
    },
    'model_hp_fixed': {
        "hidden_activation": hidden_activation,
        "final_activation": final_activation,
        "model_seed": seed_model
    },
    'optax_hp_choices': {
        'learning_rate': [0.01, 0.005, 0.003],
    },
    'optax_hp_fixed': {
        'nsteps': 300,
        'optimizer_type': 'adam',
    },
    'dl_hp_choices': {
    },
    'dl_hp_fixed': {
        'dl_seed': seed_dl,
        'num_train_sample': Ns_train,
        'num_val_sample': Ns_val,
        'batch_size': 64
    },
    'ens_seed': seed_ens,
    'training_parallel': True,
    'parallel_config': {
        'n_jobs': n_jobs, 
        'backend': 'loky',
        'verbose': 1
    },
    'device': None,
}

# Exploratory data analysis

In [11]:
# # Load the exploratory analysis result if available
# data = Data(x, y)
# data.load(f_data_save)

In [12]:
# Perform the sensitivity analysis if not done
data = Data(x, y, **data_params)
data.calculate_sensitivity(**sensitivity_params)

# Save the sensitivity analysis to disk
data.save(f_data_save)

Using the kNN-based information theoretic metrics ...
Performing pairwise analysis to remove insensitive inputs ...


  0%|          | 0/1461 [00:00<?, ?it/s]/global/cfs/cdirs/m1800/peishi/.conda/envs/kim/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
100%|██████████| 1461/1461 [21:53<00:00,  1.11it/s]  


Performing conditional independence testing to remove redundant inputs ...
Mon Nov 25 21:13:38 2024:    ERROR: auth.munge: munge_encode() failed. Socket communication error


In [13]:
# data.sensitivity_mask
data.cond_sensitivity_mask

array([[False, False, False, ..., False,  True, False],
       [False, False, False, ..., False,  True, False],
       [False, False, False, ..., False,  True, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False,  True, False, ..., False, False, False]])

# Train the inverse mapping

In [14]:
d = jax.numpy.array([0,3,5,7])
d.devices()

{CpuDevice(id=0)}

In [15]:
# Initialize three diffferent KIMs
kim1 = KIM(data, map_configs, map_option='many2many')
kim2 = KIM(data, map_configs, mask_option="sensitivity", map_option='many2one')
kim3 = KIM(data, map_configs, mask_option="cond_sensitivity", map_option='many2one')

# Train the mappings
kim1.train()
kim2.train()
kim3.train()



 Performing ensemble training in parallel with 100 model configurations...



[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
/global/cfs/cdirs/m1800/peishi/.conda/envs/kim/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
100%|██████████| 300/300 [00:03<00:00, 85.75it/s] 
[Parallel(n_jobs=20)]: Done 100 out of 100 | elapsed:  1.3min finished


Training completes.

 Performing ensemble training in parallel with 100 model configurations...



[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
100%|██████████| 300/300 [00:03<00:00, 86.17it/s] 
[Parallel(n_jobs=20)]: Done 100 out of 100 | elapsed:   17.4s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
  0%|          | 0/300 [00:00<?, ?it/s]

Training completes.

 Performing ensemble training in parallel with 100 model configurations...



...

100%|██████████| 300/300 [00:03<00:00, 91.97it/s] 
[Parallel(n_jobs=20)]: Done 100 out of 100 | elapsed:   17.1s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.


Training completes.

 Performing ensemble training in parallel with 100 model configurations...



100%|██████████| 300/300 [00:03<00:00, 94.48it/s] 


Training completes.


[Parallel(n_jobs=20)]: Done 100 out of 100 | elapsed:   48.9s finished


In [16]:
# Save 
kim1.save(f_kim_save1)
kim2.save(f_kim_save2)
kim3.save(f_kim_save3)
